## Usage
- Edit the settings in the cell below.
- Cell -> Run All.
- A Napari window will open, where you can scroll through your data.

In [10]:

# change this to point to your plate as seen in lmu_active1/instruments/Nano
plate = 'lyu/140324-A53T P62 staining/140324-A53T P62 staining/2024-03-27/20378/TimePoint_1'
plate = 'karkkael/microwell images/Plate1/2024-09-23/1/TimePoint_1/'

# set lmu_active1 root folder for Linux or Windows
#lmu_active1 = Path('/mnt/lmu_active1') # Linux
#lmu_active1 = Path('L:\lmu_active1') # Windows

# define colors you want to use (as many as you have channels)
#colormap = ["yellow", "magenta", "cyan"]
colormap = ["blue", "green", "red"]
colormap = ["blue", "green", "red", "magenta"]


## Code
You don't need to make changes in the code cells below. 

In [11]:
from aicsimageio.aics_image import AICSImage
from pathlib import Path
import matplotlib.pyplot as plt
import napari
import numpy as np
import os
import pandas as pd
import platform

def get_lmu_active1():
    current_os = platform.system()
    
    if current_os == "Windows":
        return "L:\\lmu_active1"
    elif current_os == "Linux":
        return "/mnt/lmu_active1"
    else:
        raise ValueError(f"Unsupported operating system: {current_os}")
        
# original image folder
orig = get_lmu_active1() / Path('instruments/Nano') / Path(plate)
orig = Path('/home/user/nanodata') / Path(plate)

# csv file with image sets (all channels on same row)
airflow = get_lmu_active1() / Path('airflow/nano')
plate_fixed = plate.replace(' ', '_')
csv = airflow / Path(plate_fixed) / Path('quality/Combined_MyExpt_Image.csv')
csv = '/home/user/MyExpt_Image.csv'

df = pd.read_csv(csv)
df.Metadata_Plate = df.Metadata_Plate.astype(str)
df.Metadata_Well = df.Metadata_Well.astype(str)
df.Metadata_Site = df.Metadata_Site.astype(int)

In [12]:
filename_cols = [c for c in df.columns if c.startswith('FileName')]
filename_cols

['FileName_w1', 'FileName_w2', 'FileName_w3', 'FileName_w4']

In [13]:
wavelengths = [c.replace('FileName_', '') for c in filename_cols]
wavelengths

['w1', 'w2', 'w3', 'w4']

In [14]:
metadata_cols = [c for c in df.columns if c.startswith('Metadata')]
metadata_cols

['Metadata_ChannelNumber',
 'Metadata_FileLocation',
 'Metadata_Frame',
 'Metadata_Plate',
 'Metadata_Series',
 'Metadata_Site',
 'Metadata_Well',
 'Metadata_Zslice']

In [20]:
mask = df.Metadata_Zslice.isnull()
df2d = df[mask].copy().reset_index(drop=True)
df3d = df[~mask].copy().reset_index(drop=True)
df3d.Metadata_Zslice = df3d.Metadata_Zslice.astype(int)

df2d.sort_values(by=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'], inplace=True, ignore_index=True)
df3d.sort_values(by=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site', 'Metadata_Zslice'], inplace=True, ignore_index=True)
df3d.head()

,FileName_w1,FileName_w2,FileName_w3,FileName_w4,ImageNumber,Metadata_ChannelNumber,Metadata_FileLocation,Metadata_Frame,Metadata_Plate,Metadata_Series,Metadata_Site,Metadata_Well,Metadata_Zslice,PathName_w1,PathName_w2,PathName_w3,PathName_w4
0,Plate1_C07_s2_w1CDDCF099-CA03-485B-912C-D9DEA4...,Plate1_C07_s2_w25E89F61E-80DB-431F-8344-33BEA7...,Plate1_C07_s2_w318B2E8E6-6ADC-4FE5-87F5-946A0F...,Plate1_C07_s2_w46B33AF5A-9F63-4B1B-84CC-899F0A...,19,NaN,NaN,0,Plate1,0,2,C07,9,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...
1,Plate1_C07_s2_w11F63CDB3-9ED7-4D33-8E03-F4D7ED...,Plate1_C07_s2_w25C53B106-B802-484E-8A52-AFED70...,Plate1_C07_s2_w3D77001E6-2736-4364-9EBB-22900C...,Plate1_C07_s2_w41C9C08CC-6043-4DC0-B40C-EDF5DF...,7,NaN,NaN,0,Plate1,0,2,C07,19,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...
2,Plate1_C07_s2_w11D783787-322E-450C-AF06-966AB8...,Plate1_C07_s2_w2702A70F5-7387-457D-A700-E07CEB...,Plate1_C07_s2_w3490965D9-5FDF-413A-954B-0DB737...,Plate1_C07_s2_w4EDD0B6E9-8564-47DD-B285-1BBB5E...,13,NaN,NaN,0,Plate1,0,2,C07,29,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...
3,Plate1_D06_s2_w17B7C098B-E0B9-437A-9107-D7B571...,Plate1_D06_s2_w21EB72E71-21E4-4A71-A3F9-3F8337...,Plate1_D06_s2_w367B99379-1094-4E0A-A1DC-7A799B...,Plate1_D06_s2_w44A6C20A3-DC43-4E18-96E1-1AE601...,20,NaN,NaN,0,Plate1,0,2,D06,9,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...
4,Plate1_D06_s2_w11B50CFBB-C1A7-4758-A561-EBFC0E...,Plate1_D06_s2_w20CC582FA-DCF9-49FD-B020-D91459...,Plate1_D06_s2_w373A336AC-B423-4669-9004-9F07C9...,Plate1_D06_s2_w40E3CFB9A-7793-47C5-A075-4830C3...,8,NaN,NaN,0,Plate1,0,2,D06,19,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...,/home/user/nanodata/karkkael/microwell images/...


In [22]:
df3d.groupby(by=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site']).first()

FileName_w1  \
Metadata_Plate Metadata_Well Metadata_Site                                                      
Plate1         C07           2              Plate1_C07_s2_w1CDDCF099-CA03-485B-912C-D9DEA4...   
               D06           2              Plate1_D06_s2_w17B7C098B-E0B9-437A-9107-D7B571...   
               D08           2              Plate1_D08_s2_w16FCCEB71-D49E-4FDE-A4EB-53CEFE...   
               E05           2              Plate1_E05_s2_w1D10A7B39-96B7-4967-8EC6-22EA23...   
               E08           2              Plate1_E08_s2_w1B300C335-9016-48A3-AE6D-C0FB23...   
               F08           2              Plate1_F08_s2_w1AC50FE2D-09F3-4C97-A596-15DAC7...   

                                                                                  FileName_w2  \
Metadata_Plate Metadata_Well Metadata_Site                                                      
Plate1         C07           2              Plate1_C07_s2_w25E89F61E-80DB-431F-8344-33BEA7...   
               D06           2              Plate1_D06_s2_w21EB72E71-21E4-4A71-A3F9-3F8337...   
               D08           2              Plate1_D08_s2_w202FF79CD-3141-4EED-8954-2D4E42...   
               E05           2              Plate1_E05_s2_w21929819C-D956-4C9D-9554-0BC175...   
               E08           2              Plate1_E08_s2_w243135D1D-70F0-4A28-A156-6C9437...   
               F08           2              Plate1_F08_s2_w2165FE8F0-009E-4972-9B37-3B9353...   

                                                                                  FileName_w3  \
Metadata_Plate Metadata_Well Metadata_Site                                                      
Plate1         C07           2              Plate1_C07_s2_w318B2E8E6-6ADC-4FE5-87F5-946A0F...   
               D06           2              Plate1_D06_s2_w367B99379-1094-4E0A-A1DC-7A799B...   
               D08           2              Plate1_D08_s2_w308EF5DDE-D2B7-4D3F-A818-F2BCD1...   
               E05           2              Plate1_E05_s2_w30C66A49C-E2FA-41E8-A9D7-6B9485...   
               E08           2              Plate1_E08_s2_w3CBE92F68-1A79-49C4-AFED-63E417...   
               F08           2              Plate1_F08_s2_w3F476B614-B697-4755-83AB-B15D1E...   

                                                                                  FileName_w4  \
Metadata_Plate Metadata_Well Metadata_Site                                                      
Plate1         C07           2              Plate1_C07_s2_w46B33AF5A-9F63-4B1B-84CC-899F0A...   
               D06           2              Plate1_D06_s2_w44A6C20A3-DC43-4E18-96E1-1AE601...   
               D08           2              Plate1_D08_s2_w48AD8A87B-6A41-443B-A167-FAF04F...   
               E05           2              Plate1_E05_s2_w48D523FA9-5AFA-4284-B37D-29F455...   
               E08           2              Plate1_E08_s2_w4F26499B8-C874-45A0-AE53-6912E6...   
               F08           2              Plate1_F08_s2_w43C889D94-8C40-405A-B2CA-B9CF43...   

                                            ImageNumber  \
Metadata_Plate Metadata_Well Metadata_Site                
Plate1         C07           2                       19   
               D06           2                       20   
               D08           2                       21   
               E05           2                       22   
               E08           2                       23   
               F08           2                       24   

                                            Metadata_ChannelNumber  \
Metadata_Plate Metadata_Well Metadata_Site                           
Plate1         C07           2                                 NaN   
               D06           2                                 NaN   
               D08           2                                 NaN   
               E05           2                                 NaN   
               E08           2                                 NaN   
           

In [16]:
import dask.array as da
import numpy as np

def concat_da2d(selection, filename_cols, directory):
    da_out = None
    sites = []
    for index, row in selection.iterrows():
        channels = []
        for f in filename_cols:
            path = directory / row[f]
            #print(str(path))
            img = AICSImage(path)
            #channels.append(img.data)
            channels.append(img.get_image_dask_data())
            #print(img.get_image_dask_data().shape)
        site = da.concatenate(channels, axis=2)
        sites.append(site)

    da_out = da.concatenate(sites, axis=0)
    return da_out



In [17]:
A_out = concat_da2d(df2d, filename_cols, orig)
print(A_out.shape)

(6, 1, 4, 1843, 1843)


In [19]:
from qtpy.QtWidgets import QLabel, QWidget, QVBoxLayout

viewer = napari.view_image(
        A_out,
        channel_axis=2,
        name=wavelengths,
        colormap=colormap,
        #contrast_limits=[[200, 4095], [500, 4095], [200, 4095]],
        )

# Create a custom widget to display the stack index
class StackIndexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Current stack index: 0")
        #self.label_file = QLabel("Filename w1")
        self.label_plate = QLabel("Metadata_Plate: N/A")
        self.label_well = QLabel("Metadata_Well: N/A")
        self.label_site = QLabel("Metadata_Site: N/A")
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        #layout.addWidget(self.label_file)
        layout.addWidget(self.label_plate)
        layout.addWidget(self.label_well)
        layout.addWidget(self.label_site)
        self.setLayout(layout)

    def update_index(self, index):
        self.label.setText(f"Current stack index: {index}")
        #self.label_file.setText(f"Filename w1: {df.loc[index, 'FileName_w1']}")
        self.label_plate.setText(f"Metadata_Plate: {df.loc[index, 'Metadata_Plate']}")
        self.label_well.setText(f"Metadata_Well: {df.loc[index, 'Metadata_Well']}")
        self.label_site.setText(f"Metadata_Site: {df.loc[index, 'Metadata_Site']}")

# Instantiate the custom widget
stack_index_widget = StackIndexWidget()

# Add the widget to Napari as a dock widget
viewer.window.add_dock_widget(stack_index_widget, name="Stack Index")

# Callback function to update the widget with the current stack index
def on_index_change(event):
    current_index = viewer.dims.current_step[0]
    stack_index_widget.update_index(current_index)

# Connect the callback to the dims event
viewer.dims.events.current_step.connect(on_index_change)


<function __main__.on_index_change(event)>